In [17]:
import pyspark
from pyspark.sql.functions import col, lit
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, LongType, DoubleType, TimestampType, StructField, StructType

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


In [2]:
df_green_taxi = spark.read.parquet('data/pq/green/*/*')

df_yellow_taxi = spark.read.parquet('data/pq/yellow/*/*')

In [10]:
set(df_green_taxi.columns) & set(df_yellow_taxi.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

In [8]:
df_green_taxi = df_green_taxi\
.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime')\
.withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')\

df_yellow_taxi = df_yellow_taxi\
.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')\
.withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')\



In [13]:
common_cols = []

yellow_taxi_cols = set(df_yellow_taxi.columns)

for col in df_green_taxi.columns:
    if col in yellow_taxi_cols:
        common_cols.append(col)
        


In [19]:
df_green_taxi_sel = df_green_taxi\
    .select(common_cols)\
    .withColumn('service_type', lit('green'))

df_yellow_taxi_sel = df_yellow_taxi\
    .select(common_cols)\
    .withColumn('service_type', lit('yellow'))

In [20]:
df_trips_data = df_green_taxi_sel.unionByName(df_yellow_taxi_sel)

In [23]:
df_trips_data.groupBy(col('service_type')).count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



In [24]:
df_trips_data.registerTempTable('trips_data')

/home/gabrielle/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [33]:
df_result = spark.sql('''
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
''')

In [36]:
df_result.coalesce(1).write.mode('overwrite').parquet('data/report/revenue/')